In [ ]:
import os
import csv
import time
import pandas as pd
import openai
from openai.error import APIError, ServiceUnavailableError

# Set up your OpenAI API credentials
openai.api_key = ""

file = ""#put csv file name here

SYSTEM_MESSAGE = [
    "You are a helpful virtual assistant with knowledge regarding the specific concern of privacy when it comes to virtual reality. ",
    "You are capable of analyzing comments and determining whether they are expressing concern specifically with virtual reality privacy",
    "You are capable of assigning themes of specifically virtual reality privacy concerns when to a comment if it is expressing a privacy concern",
]

CHAT_HISTORY = [
    {"role": "system", "content": message} for message in SYSTEM_MESSAGE
]

def talkToChatGPT(new_message):
    message_to_append = {"role": "user", "content": new_message}
    CHAT_HISTORY.append(message_to_append)
    
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        max_tokens=150,
        messages=CHAT_HISTORY,
    )

    chatGPT_reply = completion.choices[0].message.content
    CHAT_HISTORY.append({"role": "assistant", "content": chatGPT_reply})
    return chatGPT_reply, CHAT_HISTORY

def clearContext():
    global CHAT_HISTORY
    CHAT_HISTORY = [
        {"role": "system", "content": message} for message in SYSTEM_MESSAGE
    ]

def write_to_csv(filename, headers, content, folder_name):
    folder_path = os.path.join(os.getcwd(), folder_name)  
    os.makedirs(folder_path, exist_ok=True)  
    
    file_path = os.path.join(folder_path, filename)
    
    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            writer.writerow(headers)
        writer.writerow(content)

filename = str(file)+".csv"
chunk_size = 100
woYes= 'comments_without_yes_'+str(file)+".csv"#writes comments wo yes to a seperate csv file

chunk_number = 0

running = True

while running:
    try:
        for chunk in pd.read_csv(filename, chunksize=chunk_size):
            comments = chunk["comment"].values.tolist()

            for user_comment in comments:
                try:
                    query = f"Parse through this comment and determine if it is concerned with specifically the privacy of virtual reality. Answer with only Yes or No:  {user_comment}"
                    result, CHAT_HISTORY = talkToChatGPT(query)
                    
                    if 'yes' in result.lower():
                        themes_query = "Identify any themes related to specifically virtual reality privacy concerns in only the previous comment. Provide the answer in this format Theme: "
                        themes, CHAT_HISTORY = talkToChatGPT(themes_query)
                        evidence_query = "Provide direct quotational evidence from only the previous comment to support each theme you provided. Provide the answer in this format Evidence: "
                        evidence, CHAT_HISTORY = talkToChatGPT(evidence_query)
                        write_to_csv('ThemesandQuotevirtualreality.csv', ['themes', 'quotational evidence'], [themes, evidence], str(file))
                    else:
                        write_to_csv(woYes, ['Did not give a yes'], [user_comment], str(file))

                    clearContext()

                except (APIError, ServiceUnavailableError) as e:
                    print("An error occurred:", str(e))
                    print("Sleeping for 60 seconds before retrying...")
                    time.sleep(60)
                    continue

                except Exception as e:
                    print("An unexpected error occurred:", str(e))
                    print("Sleeping for 60 seconds before retrying...")
                    time.sleep(60)
                    continue

            chunk_number += 1
            print(chunk_number)
        running = False

    except KeyboardInterrupt:
        print("Execution stopped by user")
        running = False
